In [1]:
import configparser
import os

from dotenv import load_dotenv

load_dotenv(".env")

CONFIG = configparser.ConfigParser()
PATH_CONFIG = os.getenv('path_2_config')

ENVIRONMENT = os.getenv('environment')
if ENVIRONMENT=="windows":
    CONFIG.read(PATH_CONFIG, encoding='utf-8')
else:
    CONFIG.read(PATH_CONFIG)

Python-dotenv could not parse statement starting at line 2


## 超參數設定


### 停止條件

In [2]:
PATH_STOP_FILE = os.path.join(os.getcwd(), 'stop_true.txt') # 人工 early stop 的檔案位置
NUM_NEW_PROMPT = 36   # 必須是 prompt_population 的倍數
NUM_EXPERIMENT = 9  # 實驗次數
STOP_SCORE=150  # 練蠱終止條件(我是設超過baseline做100題後的分數，這邊看你資料量來設)

### 路徑

In [3]:
from pathlib import Path

folder_record = Path(CONFIG["datapath"]["record_folder"])
name_experiment = "experiment_7" # 主要是改這行
FOLDER_EXPERIMENT = folder_record / name_experiment

# PATH_INIT = "/user_data/intern/Ress/init_prompt/2024_0731_1313_data_150-30.json"
PATH_INIT = "/user_data/intern/Ress/init_prompt/test.json"

## 製作初始化資料

**只有要建立新資料的時候才需要跑**

In [3]:
from main import init_setting

type_llm = "Breeze"
# type_embedding = "multi-qa-mpnet-base-dot-v1"
type_embedding = "bgem3"
# type_embedding = None

path_prompt = CONFIG["datapath"]["init_os_prompt"]
# path_prompt = "D:\\實習\\evoprompt\\Ress\\dataset\\init_os_prompt_corpus_2024_0724_1357.json"
# path_prompt = "D:\\實習\\evoprompt\\Ress\\dataset\\init_os_prompt_corpus.json"

path_data = CONFIG["datapath"]["Final_Quality"]
print(f"{path_data=}")

ttl_model, ttl_dataset, ttl_pair_os_prompt_scores = init_setting(type_llm, type_embedding, path_data, path_prompt)

Python-dotenv could not parse statement starting at line 2


path_data='/user_data/intern/Ress/dataset/Final_Quality.json'


  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [4]:
import json
from utils.tools import get_file_name, time_now

t = time_now()
path_folder = "/user_data/intern/Ress/init_prompt"
path_file = f"{path_folder}/{t}_data_150-30.json"
data = {
    "corpus": get_file_name(path_data),
    "type_llm": type_llm,
    "type_embedding": type_embedding,
    "prompt_popularion": ttl_pair_os_prompt_scores,
    "data": ttl_dataset
}
with open(path_file, 'w') as file:
    json.dump(data, file, indent=4)

NameError: name 'path_data' is not defined

## 初始化

In [4]:
import json

from utils.call_model.embedding import Encoder
from utils.call_model.llm import LLM

def get_init(path_init=PATH_INIT):
    
    with open(path_init,  'r') as file:
        record_init = json.load(file)

    # 指定 llm
    llm = LLM(record_init["type_llm"])

    # 指定 embedding model
    if record_init["type_embedding"] is None:
        embedding_model = None
    else:
        embedding_model = Encoder(record_init["type_embedding"])

    ttl_model = (llm, embedding_model)
    
    return ttl_model, record_init

/user_data/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# 實驗

test_EvoDE 跟 test_EvoGA 的架構一模一樣，但為了方便 debug，所以沒有另外建 function

test_ReSS 跟前兩者只差在存檔次數不同<br>
前兩者每個 population 都會存，test_ReSS 則只存一次

## ReSS

In [5]:
import json
from utils.ReSS import ReSS
from utils.tools import time_now

def test_ReSS():
    """ 每生成一個 prompt 就存一次 population
    """
    
    # init
    ttl_model, record_init = get_init()
    corpus = record_init["corpus"]
    type_llm = record_init["type_llm"]
    type_embedding = record_init["type_embedding"]
    ttl_pair_os_prompt_scores = record_init["prompt_popularion"]
    ttl_dataset = record_init["data"]

    # 停止條件
    stop_run_num = NUM_NEW_PROMPT   # 或是設一個回合數來終止(本來我會讓他跑到天荒地老所以沒有用for loop)
    
    # 跑實驗
    record_population = []
    record_population.append(ttl_pair_os_prompt_scores) # 起始狀態
    
    sorted_pair = sorted(ttl_pair_os_prompt_scores,  key=lambda x: x['train_score'],  reverse=True)
    while(
        sorted_pair[0]['train_score']<STOP_SCORE
        and stop_run_num>0
        and not os.path.exists(PATH_STOP_FILE)   # 人工 early stop
    ):
        new_population = ReSS(ttl_model, ttl_dataset, sorted_pair)
        
        record_population.append(new_population)
        sorted_pair = sorted(new_population,  key=lambda x: x['train_score'],  reverse=True)

        stop_run_num -= 1

    # 儲存結果
    t = time_now()
    file_path = FOLDER_EXPERIMENT / f"{t}_ReSS.json"
    data = {
        "corpus": corpus,
        "type_llm": type_llm,
        "type_embedding": type_embedding,
        "best_promt": sorted_pair[0],
        "record": record_population
    }
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
    print(f"\n\n\nthe result is saved at:\n{file_path}")

In [6]:
for i in range(NUM_EXPERIMENT):
    test_ReSS()

  0%|          | 0/150 [00:00<?, ?it/s]

KeyboardInterrupt: 

## EvoDE

In [5]:

import json
from utils.EvoPrompt import EvoDE
from utils.tools import time_now

def test_EvoDE():
    """ 每更新一次整個 population 才儲存一次
    """
    
    # 因為
    t = time_now()
    path_folder = FOLDER_EXPERIMENT / f"{t}_Evo"
    path_folder.mkdir(parents=True, exist_ok=True)
    
    # init
    ttl_model, record_init = get_init()
    corpus = record_init["corpus"]
    type_llm = record_init["type_llm"]
    type_embedding = record_init["type_embedding"]
    ttl_pair_os_prompt_scores = record_init["prompt_popularion"]
    ttl_dataset = record_init["data"]
    
    ttl_dataset = {
        "train_split": ttl_dataset["train_split"][:2],
        "dev_split": ttl_dataset["dev_split"][:2]
    }

    # 停止條件
    stop_run_num = int(NUM_NEW_PROMPT/4)   # 或是設一個回合數來終止(本來我會讓他跑到天荒地老所以沒有用for loop)
    
    # 跑實驗
    record_population = []
    record_population.append(ttl_pair_os_prompt_scores) # 起始狀態
    
    sorted_pair = sorted(ttl_pair_os_prompt_scores,  key=lambda x: x['train_score'],  reverse=True)
    while(
        sorted_pair[0]['train_score']<STOP_SCORE
        and stop_run_num>0
        and not os.path.exists(PATH_STOP_FILE)   # 人工 early stop
    ):
        new_population = EvoDE(ttl_model, ttl_dataset, sorted_pair)
        
        record_population.append(new_population)
        sorted_pair = sorted(new_population,  key=lambda x: x['train_score'],  reverse=True)

        # 儲存結果
        file_path = path_folder / f"{int(NUM_NEW_PROMPT/4) - stop_run_num:02d}_EvoDE.json"
        data = {
            "corpus": corpus,
            "type_llm": type_llm,
            "type_embedding": type_embedding,
            "best_promt": sorted_pair[0],
            "record": sorted_pair
        }
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"\n\n\nthe result is saved at:\n{file_path}")
        
        stop_run_num -= 1
    

In [6]:
pre_file_path = None
for i in range(NUM_EXPERIMENT):
    test_EvoDE()

  0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

## EvoGA

In [5]:
import json
from utils.EvoPrompt import EvoGA
from utils.tools import time_now

def test_EvoGA():
    """ 每更新一次整個 population 才儲存一次
    """
    t = time_now()
    path_folder = FOLDER_EXPERIMENT / f"{t}_EvoGA"
    path_folder.mkdir(parents=True, exist_ok=True)
    
    # init
    ttl_model, record_init = get_init()
    corpus = record_init["corpus"]
    type_llm = record_init["type_llm"]
    type_embedding = record_init["type_embedding"]
    ttl_pair_os_prompt_scores = record_init["prompt_popularion"]
    ttl_dataset = record_init["data"]
    
    ttl_dataset = {
        "train_split": ttl_dataset["train_split"][:2],
        "dev_split": ttl_dataset["dev_split"][:2]
    }

    # 停止條件
    stop_run_num = int(NUM_NEW_PROMPT/4)   # 或是設一個回合數來終止(本來我會讓他跑到天荒地老所以沒有用for loop)

    # 跑實驗
    record_population = []
    record_population.append(ttl_pair_os_prompt_scores)
    
    sorted_pair = sorted(ttl_pair_os_prompt_scores,  key=lambda x: x['train_score'],  reverse=True)
    while(
        sorted_pair[0]['train_score']<STOP_SCORE
        and stop_run_num>0
        and not os.path.exists(PATH_STOP_FILE)   # 人工 early stop
    ):
        new_population = EvoGA(ttl_model, ttl_dataset, sorted_pair)
        
        record_population.append(new_population)
        sorted_pair = sorted(new_population,  key=lambda x: x['train_score'],  reverse=True)

        # 儲存結果
        file_path = path_folder / f"{int(NUM_NEW_PROMPT/4) - stop_run_num:02d}_EvoGA.json"
        data = {
            "corpus": corpus,
            "type_llm": type_llm,
            "type_embedding": type_embedding,
            "best_promt": sorted_pair[0],
            "record": sorted_pair
        }
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"\n\n\nthe result is saved at:\n{file_path}")
        
        stop_run_num -= 1
    

"統一GA跟DE的格式

In [6]:
pre_file_path = None
for i in range(NUM_EXPERIMENT):
    test_EvoGA()

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 